<a href="https://colab.research.google.com/github/naufalhisyam/TurbidityPrediction-thesis/blob/main/train_model_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
!pip install tensorflow-addons
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [ ]:
!git clone https://github.com/naufalhisyam/TurbidityPrediction-thesis.git
os.chdir('/content/TurbidityPrediction-thesis') 

**PREPARING DATASET**

In [ ]:
images = pd.read_csv(r'./Datasets/0degree/0degInfo.csv') #load dataset info
train_df, test_df = train_test_split(images, train_size=0.9, shuffle=True, random_state=1) #Split into train and test set

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

**CREATING THE MODEL**

Model Architecture

In [ ]:
def get_model():
    #Create model
    base_model = tf.keras.applications.DenseNet121(
        include_top=False, weights=None, 
        input_shape=(224, 224, 3), pooling='avg')
    out = base_model.output
    prediction = tf.keras.layers.Dense(1, activation="linear")(out)
    model = tf.keras.Model(inputs = base_model.input, outputs = prediction)

    #Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss="mse", optimizer=opt,
                  metrics=['mae', tfa.metrics.RSquare(name="R2")])
    
    return model

model = get_model()
tf.test.gpu_device_name()

Training Callbacks

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
        
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5,
    restore_best_weights=True)

class CustomModelCheckpointCallback(tf.keras.callbacks.ModelCheckpoint):

    def __init__(self, ignore_first, *args, **kwargs):
      super(CustomModelCheckpointCallback, self).__init__(*args, **kwargs)
      self.ignore_first = ignore_first

    def on_epoch_end(self, epoch, logs):
        if epoch+1> self.ignore_first:
            super().on_epoch_end(epoch, logs)

pathname = 'saved_model/densenet-epoch{epoch:02d}-loss{val_loss:.2f}'
checkpoint = CustomModelCheckpointCallback(
    ignore_first=80, filepath = pathname,
    monitor='val_loss', mode='min',
    save_best_only=True, save_freq='epoch')

Training Model

In [ ]:
num_epoch = 100
history = model.fit(train_images, validation_data=val_images, 
                    epochs=num_epoch, batch_size=8, callbacks=[tensorboard_callback, checkpoint], verbose=1)

Save Model Manually

In [ ]:
last_val_loss = history.history['val_loss'][-1]
name = f'densenet-epoch{num_epoch}-loss{last_val_loss}'
model.save(f"saved_model/{name}")
hist_df = pd.DataFrame(history.history)
hist_csv_file = f'saved_model/{name}/history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
#tf.keras.utils.plot_model(model, f"saved_model/{name}/densenet_model_arch.png", show_shapes=False)

Model Evaluation

In [ ]:
%tensorboard --logdir logs

In [ ]:
pred_turbid = np.squeeze(model.predict(test_images))
true_turbid = test_images.labels
residuals = true_turbid - pred_turbid

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(8,5), gridspec_kw={'width_ratios': [4, 1]})

axs[0].scatter(pred_turbid,residuals)
axs[0].set_title('Residual Plot dari Model DenseNet-121', fontsize=13, fontweight='bold')           
axs[0].set_ylabel('Residual')
axs[0].set_xlabel('Predicted Turbidity')      
axs[0].axhline(0, color='black')
axs[0].grid()

axs[1].hist(residuals, bins=40, orientation="horizontal", density=True)
axs[1].axhline(0, color='black')
axs[1].set_xlabel('Distribution')  
axs[1].yaxis.tick_right()
axs[1].grid(axis='y')

plt.subplots_adjust(wspace=0.05)
plt.savefig(f'saved_model/{name}/residualPlot_{name}.png', dpi=150)
plt.show()

In [ ]:
ms_error = history.history['loss']
val_ms_error = history.history['val_loss']
ma_error = history.history['mae']
val_ma_error = history.history['val_mae']
r2 = history.history['R2']
val_r2 = history.history['val_R2']

epochs = range(1, len(ms_error) + 1)

f, axs = plt.subplots(3, 1, figsize=(6,14))
axs[0].plot(epochs, ms_error, 'tab:orange', label='train_loss (mse)')
axs[0].plot(epochs, val_ms_error, 'tab:blue', label='val_loss (mse)')
axs[0].set_title('MSE Selama Training', fontsize=13, fontweight='bold')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('MSE')
axs[0].legend(facecolor='white')
axs[0].grid()

axs[1].plot(epochs, ma_error, 'tab:orange', label='train_mae')
axs[1].plot(epochs, val_ma_error, 'tab:blue', label='val_mae')
axs[1].set_title('MAE Selama Training', fontsize=13, fontweight='bold')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('MAE')
axs[1].legend(facecolor='white')
axs[1].grid()

axs[2].plot(epochs, r2, 'tab:orange', label='train_R2')
axs[2].plot(epochs, val_r2, 'tab:blue', label='val_R2')
axs[2].set_title('$R^2$ Selama Training', fontsize=13, fontweight='bold')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('$R^2$')
axs[2].legend(facecolor='white')
axs[2].grid()

plt.tight_layout()
plt.savefig(f'saved_model/{name}/trainPlot_{name}.png', dpi=150)
plt.show()

Copy to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

save_path = f"/content/gdrive/MyDrive/Hasil_Training/DenseNet"
if not os.path.exists(save_path):
  os.makedirs(save_path)

oripath = "saved_model/."
!cp -a "{oripath}" "{save_path}" # copies files to google drive